### 0. Install Julia and POMDP.jl

Please first install Julia and POMDP.jl (https://github.com/JuliaPOMDP/POMDPs.jl).


### 1. Include Julia packages and load the problem
Several POMDP julia packages such as "POMDPTools, POMDPModels" are already included in planner.jl.

If there is an error about uninstalled packages, please install them with "pkg.add()".

In [ ]:
include("./ContinuousPOMCGS/Planner.jl")
using RoombaPOMDPs

# --- Roomba Lidar---
num_x_pts = 25 # or 41
num_y_pts = 16 # or 26
num_th_pts = 10 # or 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
pomdp = RoombaPOMDP(sensor=Lidar(), mdp=RoombaMDP(config=3, sspace=sspace))

# Note that actions(pomdp) and rand(actions(pomdp)) is not properly implemented in the current RoombaPOMDP.jl package
# Thus, one has to either implement those two functions in RoombaPOMDPs to sample an action from the continous action space, or using the following configuration
# In this example, we represent continuous ranges with a large number of discrete actions (1071 actions)
max_speed = 5.0
speed_interval = 0.1
max_turn_rate = 1.0
turn_rate_interval = 0.1
action_space = vec([RoombaAct(v, om) for v in 0:speed_interval:max_speed, om in -max_turn_rate:turn_rate_interval:max_turn_rate])
pomdp = RoombaPOMDP(sensor=Lidar(), mdp=RoombaMDP(config=3, aspace=action_space, v_max=max_speed, sspace=sspace))

### 2. Define parameters

In [ ]:
## This is an exmaple parameter configuraiton ##
b0 = initialstate(pomdp) # Get POMDP's initial belief
nb_process_samples = 10000
max_b_gap = 0.3
nb_particles_b0 = nb_process_samples
c = 1.0
nb_abstract_obs = 5
epsilon = 0.1
particles_b0, dict_weighted_b0 = InitContinuousActionPOMDP(pomdp, b0, nb_particles_b0)
rmin, a_rmin = FindRLower(pomdp, b0, action_space)

### 3. Prepare $V_{mdp}$

Here we use a Q-learning algorithm to compute $V_{MDP}$, one can also use other MDP solvers.

In [ ]:
Q_table = Dict{Any, Dict{Int64, Float64}}()
V_table = Dict{Any, Float64}()
learning_rate = 0.9
explore_rate = 0.7
# Define the size of the subset action space for Q-learning
subset_size = 20
# Randomly choose a subset without replacement
subset_actions = sample(action_space, subset_size, replace=false)
Vmdp = Qlearning(Q_table, V_table, learning_rate, explore_rate, subset_actions, typemin(Float64), typemax(Float64))
nb_episode_size = 10
nb_max_episode = 5
nb_sim = 10
Training(Vmdp, nb_episode_size, nb_max_episode, nb_sim, epsilon, particles_b0, pomdp)

### 4. Init POMCGS and run the solver 

In [ ]:
# --- 1: Define POMCGS and fsc ---
planner = ContinuousPlannerPOMCGS(rmin, Vmdp)
planner._bool_APW = true # using action progressive widening
InitPlannerParameters(planner, 
                    nb_process_samples, 
                    max_b_gap,
                    c,
                    discount(pomdp),
                    # nb_abstract_samples,
                    nb_abstract_obs,
                    false,
                    Vector{Float64}(),
                    epsilon)
fsc = InitFSC(max_b_gap, planner._max_graph_node_size, action_space)

In [ ]:
# --- 2: Graph Search ---
Search(pomdp, particles_b0, dict_weighted_b0, fsc, planner)

### 5. Simulate the computed FSC

In [ ]:
SimulationFSC(b0, pomdp, fsc, discount(pomdp), planner)

### 6. Evaluate the computed FSC

In [ ]:
nb_eval = 100000
EvaluationWithSimulationFSC(pomdp, fsc, discount(pomdp), nb_eval, planner)